# Wikipedia

## Una matriz de frecuencia de palabras tf-idf
Empezaremos creando una matriz de frecuencia de palabras `tf-idf` para una colección de documentos. Para ello, utilice el `TfidfVectorizer`. Transforma una lista de documentos en una matriz de frecuencias de palabras, cuya salida es una matriz `csr_matrix`.

In [ ]:
# Crear un TfidfVectorizer: tfidf
# tfidf = TfidfVectorizer() 

# Aplicar fit_transform al documento: csr_mat
# csr_mat = tfidf.fit_transform(documents)

# Imprimir el resultado del método toarray()
# print(csr_mat.toarray())

# Obtener las palabras: words
# words = tfidf.get_feature_names()

# Imprimir palabras
# print(words)

## Agrupación de Wikipedia parte I 

Recordemos que `TruncatedSVD` puede realizar PCA en matrices dispersas en formato `csr_matrix`, como matrices de frecuencia de palabras. 

Combinemos `TruncatedSVD` y `k-means` para agrupar algunas páginas populares de Wikipedia. 

In [105]:
# LIBRERIAS
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import NMF
from sklearn.preprocessing import normalize

In [96]:
# DATA
ruta_archivo = '/home/jovyan/notebooks/data/wikipedia-vectors.csv'
articles = pd.read_csv(ruta_archivo,index_col = 0)
titles = articles.columns.tolist()
articles = articles.values
articles = articles.T
print(len(titles))
print(articles.shape)
ruta_archivo = '/home/jovyan/notebooks/data/wikipedia-vocabulary-utf8.txt'
with open(ruta_archivo, 'r') as archivo:
    # Lee las líneas del archivo
    words = [linea.strip() for linea in archivo]

60
(60, 13125)


In [71]:
# Crear una instancia TruncatedSVD: svd
svd = TruncatedSVD(n_components = 50)

# Crear una instancia KMeans: kmeans
kmeans = KMeans(n_clusters = 6, n_init = 10)

# Crear un pipeline: pipeline
pipeline = make_pipeline(svd, kmeans)

## Wikipedia en clústeres, parte II


Recordemos que tenemos una matriz de artículos de tf-idf palabra-frecuencia de algunos artículos populares de Wikipedia, y una lista de títulos `titles. Utilizaremos el pipeline para agrupar los artículos de Wikipedia.

In [73]:
# Ajustar el pipeline a los artículos
pipeline.fit(articles)

# Predecir las etiquetas de los clusters: labels
labels = pipeline.predict(articles)

# Crear un DataFrame alineando las etiquetas y los títulos: df
df = pd.DataFrame({'label': labels, 'article': titles})

# Mostrar df ordenado por la etiqueta del cluster
print(df.sort_values('label'))

    label                                        article
14      0                                 Climate change
19      0  2007 United Nations Climate Change Conference
18      0  2010 United Nations Climate Change Conference
17      0  Greenhouse gas emissions by the United States
16      0                                        350.org
15      0                                 Kyoto Protocol
13      0                               Connie Hedegaard
12      0                                   Nigel Lawson
11      0       Nationally Appropriate Mitigation Action
10      0                                 Global warming
59      1                                    Adam Levine
50      1                                   Chad Kroeger
57      1                          Red Hot Chili Peppers
56      1                                       Skrillex
55      1                                  Black Sabbath
54      1                                 Arctic Monkeys
58      1                      

## NMF aplicado a artículos de Wikipedia

Ajustaremos el modelo y transforma los artículos.

In [75]:
# Crear una instancia de NMF: model
model = NMF(n_components = 6)

# Ajustar el modelo a los artículos
model.fit(articles)

# Transformar los artículos: nmf_features
nmf_features = model.transform(articles)

# Imprimir las características de NMF
print(nmf_features.round(2))

[[0.   0.   0.   0.   0.   0.44]
 [0.   0.   0.   0.   0.   0.56]
 [0.   0.   0.   0.   0.   0.4 ]
 [0.   0.   0.   0.   0.   0.38]
 [0.   0.   0.   0.   0.   0.48]
 [0.01 0.01 0.01 0.03 0.   0.33]
 [0.   0.   0.02 0.   0.01 0.36]
 [0.   0.   0.   0.   0.   0.49]
 [0.02 0.01 0.   0.02 0.03 0.48]
 [0.01 0.03 0.03 0.07 0.02 0.34]
 [0.   0.   0.53 0.   0.03 0.  ]
 [0.   0.   0.35 0.   0.   0.  ]
 [0.01 0.01 0.31 0.06 0.01 0.02]
 [0.   0.01 0.34 0.01 0.   0.  ]
 [0.   0.   0.43 0.   0.04 0.  ]
 [0.   0.   0.48 0.   0.   0.  ]
 [0.01 0.02 0.37 0.03 0.   0.01]
 [0.   0.   0.48 0.   0.   0.  ]
 [0.   0.01 0.55 0.   0.   0.  ]
 [0.   0.   0.46 0.   0.   0.  ]
 [0.   0.01 0.02 0.51 0.06 0.01]
 [0.   0.   0.   0.51 0.   0.  ]
 [0.   0.01 0.   0.42 0.   0.  ]
 [0.   0.   0.   0.43 0.   0.  ]
 [0.   0.   0.   0.49 0.   0.  ]
 [0.1  0.09 0.   0.38 0.   0.01]
 [0.   0.   0.   0.57 0.   0.01]
 [0.01 0.01 0.   0.47 0.   0.01]
 [0.   0.   0.   0.57 0.   0.  ]
 [0.   0.   0.   0.52 0.01 0.01]
 [0.   0.4

## Características NMF de los artículos de Wikipedia

Ahora exploraremos las características NMF que creamos.

Al investigar las características, observemos que para ambos actores, la característica 3 NMF tiene con diferencia el valor más alto. Esto significa que ambos artículos se reconstruyen utilizando principalmente el componente 3 del NMF.

In [76]:
# Crear un DataFrame de pandas: df
df = pd.DataFrame(nmf_features, index = titles)

# Imprimir la fila para 'Anne Hathaway'
print(df.loc["Anne Hathaway"])

# Imprimir la fila para 'Denzel Washington'
print(df.loc["Denzel Washington"])

0    0.003815
1    0.000000
2    0.000000
3    0.571957
4    0.000000
5    0.000000
Name: Anne Hathaway, dtype: float64
0    0.000000
1    0.005575
2    0.000000
3    0.419630
4    0.000000
5    0.000000
Name: Denzel Washington, dtype: float64


## El NMF aprende los temas de los documentos

Cuando se aplica NMF a documentos, los componentes corresponden a temas de documentos, y las características NMF reconstruyen los documentos a partir de los temas. Comprobaremos esto con el modelo NMF que construimos anteriormente utilizando los artículos de Wikipedia. 

Anteriormente, viste que el valor de la tercera característica NMF era alto para los artículos sobre los actores Anne Hathaway y Denzel Washington. Acá identificaremos el tema del componente NMF correspondiente.

El modelo NMF que construyó anteriormente está disponible como modelo, mientras que palabras es una lista de las palabras que etiquetan las columnas de la matriz de frecuencia de palabras.

Cuando hayas terminado, tómate un momento para reconocer el tema que tienen en común los artículos sobre Anne Hathaway y Denzel Washington.

In [103]:
# Crear un DataFrame components_df a partir de model.components_, 
# estableciendo columns=words para que las columnas estén etiquetadas 
# por las palabras.
components_df = pd.DataFrame(model.components_, columns = words)

# Imprimir la forma del DataFrame
print(components_df.shape)

# Seleccionar la fila 3: component
component = components_df.iloc[3]

# Llamar al método .nlargest() e imprima el resultado. 
## Esto da las cinco palabras con los valores más altos 
## para ese componente.
print(component.nlargest())

(6, 13125)
film       0.632005
award      0.254794
starred    0.246897
role       0.212841
actress    0.187623
Name: 3, dtype: float64


## ¿Qué artículos son similares a "Cristiano Ronaldo"?
Ahora, encontraremos los artículos más similares al artículo sobre el futbolista Cristiano Ronaldo. 

In [106]:
# Normalizar las características de NMF: norm_features
norm_features = normalize(nmf_features)

# Crear un DataFrame: df
df = pd.DataFrame(norm_features, index = titles)

# Seleccionar la fila correspondiente a 'Cristiano Ronaldo': article
article = df.loc['Cristiano Ronaldo']

# Calcular los productos punto: similarities
similarities = df.dot(article)

# Mostrar aquellos con la mayor similitud del coseno
print(similarities.nlargest())

Cristiano Ronaldo                1.000000
Franck Ribéry                    0.999973
Radamel Falcao                   0.999942
Zlatan Ibrahimović               0.999942
France national football team    0.999923
dtype: float64
